In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

In [ ]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
default_config = utils.get_defaults_config()


utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation



network_name = "caterpillar_standard_3"
dcn = dce.load_network_from_config(network_name)
dcn.eval()
dataset = dcn.load_training_dataset()

In [ ]:
from PIL import Image
from plotting import normalize_descriptor
import time

## Usage:
## change the full_rgb_only_path
## run the full notebook

full_rgb_only_path = "/media/peteflo/3TBbackup/dense_correspondence/pdc/logs_proto/2018-04-27-19-39-51/processed/rgb_only"
save_path = os.path.join(full_rgb_only_path, "../descriptors_only_masked")
os.system("mkdir -p "+ save_path)

descriptor_image_stats = dcn.descriptor_image_stats

for i in sorted(os.listdir(full_rgb_only_path)):
    start = time.time()
    if "rgb.png" not in i:
        continue

    rgb_filename = os.path.join(full_rgb_only_path, i)
    rgb_a = Image.open(rgb_filename).convert('RGB')

    # compute dense descriptors
    # This takes in a PIL image!
    rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)

    # these are Variables holding torch.FloatTensors, first grab the data, then convert to numpy
    res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()

    res_a = normalize_descriptor(res_a, descriptor_image_stats["mask_image"])
    save_file_name = os.path.join(save_path, i)
    plt.imsave(save_file_name, res_a)
    print "forward and saving at rate", time.time()-start


plt.imshow(res_a)
plt.show()

In [ ]:
# best so far
# ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -r 30 out.mp4

In [ ]:
os.system("cd "+save_path+" && ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -r 30 out.mp4")

In [ ]:
# This currently doesn't work
# import cv2
# import numpy as np

# fourcc = cv2.VideoWriter_fourcc(*'DIVX')  # 'x264' doesn't work
# out = cv2.VideoWriter('./videos/001_output.mp4',fourcc, 29.0, size, False)  # 'False' for 1-ch instead of 3-ch for color
# fgbg= cv2.createBackgroundSubtractorMOG2()

# while (capture.isOpened()):  #while Ture:
#     ret, img = capture.read()
#     if ret==True:
#         fgmask = fgbg.apply(img)
#         out.write(fgmask)
#         cv2.imshow('img',fgmask)

#     #if(cv2.waitKey(27)!=-1):  # observed it will close the imshow window immediately
#     #    break                 # so change to below
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# capture.release()
# out.release()
# cv2.destroyAllWindows()